In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from scipy.interpolate.interpolate import interp1d
import plotly.graph_objects as go

In [2]:
class DataHandler:
    def __init__(self, spinner=False, blades=3):
        self.spinner = 's' if spinner else ''
        self.blades = blades
        self.load_data()

    def filename(self):
        return 'clark_{}{}_cp.csv'.format(self.blades, self.spinner)
    
    def load_data(self):
        data = pd.read_csv('data\\large_angles\\' + self.filename(), engine='python')
        data = data.melt(id_vars = ['x'], var_name = 'y', value_name = 'z').dropna()
        data.y = data.y.astype(float)
        self.data = data.reset_index(drop=True)
        
    def save_data(self):
        df_pivot = self.curves.pivot(index='y', columns='x', values='z')
        df_pivot = df_pivot.iloc[::-1].fillna(value=0, limit=2).iloc[::-1]
        df = pd.melt(df_pivot.reset_index(), id_vars='y', value_vars=df_pivot.columns, value_name = 'z')
        self.curves = df[['x', 'y', 'z']].dropna().reset_index(drop=True).sort_values(by=['x', 'y'])
        df_pivot.to_csv('data\\mesh\\pivot\\' + self.filename(), index=False)
        self.curves.to_csv('data\\mesh\\' + self.filename(), index=False)
              
    def appendTo(self, df, x, y, z):
        return df.append(pd.DataFrame({'x': x, 'y': y, 'z': z}), ignore_index=True)
        
        

In [3]:
class Propeller(DataHandler):
    
    def plot_data(self, df, color, name, size=3):
        return go.Scatter3d(
            x=df.x, y=df.y, z=df.z,
            mode='markers',
            marker=dict(size=size, color=color, opacity=0.9),
            name=name)
    
    def layout(self):
        camera = dict(eye=dict(x=1.3, y=-1.3, z=0.5),
                      center=dict(y=-0.1, z=-0.15))
        scene = dict(
            xaxis = dict(title='J'),
            yaxis = dict(title='Angle'),
            zaxis = dict(title='Cp')
        )
        legend = dict(font = {'size': 12}, itemsizing = 'trace')
        return go.Layout(margin=dict(l=0, r=0, b=0, t=0), 
                         width=1024, height=768, 
                         scene=scene,
                         scene_camera=camera,
                         legend_title_text='Data',
                         legend=legend)
    
    def draw(self, y_max=60):
        df = self.data[self.data.y <= y_max]
        fig = go.Figure(layout = self.layout())
        fig.add_trace(self.plot_data(df, 'black', 'NACA'))
        if hasattr(self, 'extra') and self.data.min().y == 15:
            fig.add_trace(self.plot_data(self.extra, 'gray', 'New', 4))
        if hasattr(self, 'mesh'):
            df = self.mesh[self.mesh.y <= y_max]
            fig.add_trace(self.plot_data(df, 'red', 'Sides'))
        if hasattr(self, 'top'):
            df = self.top[self.top.y <= y_max]
            fig.add_trace(self.plot_data(df, 'green', 'Top'))
        fig.show()
    
    def drawXZ(self):
        fig = go.Figure(layout = self.layout())
        fig.add_trace(self.plot_data(self.data, 'black', 'NACA'))
        if hasattr(self, 'meshXZ'):
            fig.add_trace(self.plot_data(self.meshXZ, 'gray', 'New'))
        fig.show()
        
    def draw_curves(self, y_max=60, top=True):
        df = self.curves[self.curves.y <= y_max].reset_index(drop=True)
        fig = go.Figure(layout = self.layout())
        fig.add_trace(self.plot_data(df, 'black', 'Siatka', 2))
        if top:
            df = self.top[self.top.y <= y_max].reset_index(drop=True)
            fig.add_trace(self.plot_data(df, 'green', 'Góra'))
        fig.show()

    def draw_surface(self):
        data = self.curves.drop_duplicates(subset=['x', 'y'])
        data = data.pivot(index='y', columns='x', values='z')
        fig = go.Figure(go.Surface(
            z=data.values,
            x=data.columns,
            y=data.index,
            showscale=False,
            colorscale='algae',
            opacity=0.9,
            reversescale=True,
            hovertemplate=
            '<b>J</b>: %{x}<br>' +
            '<b>Angle</b>: %{y}°<br>' +
            '<b>Cp</b>: %{z}<extra></extra>',
        ), layout = self.layout())
        fig.show()
        
    def fit(self):
        fig = go.Figure(layout=go.Layout(margin=dict(l=20, r=20, b=0, t=0), width=240, height=480, showlegend=False))
        points = self.top[self.top.y==10]
        fig.add_trace(go.Scatter(x=points.x, 
                                 y=points.z, 
                                 mode='markers', 
                                 marker=dict(size=5, color='green')))
        point = self.mesh[self.mesh.y==10]
        fig.add_trace(go.Scatter(x=point.x, 
                                    y=point.z, 
                                    mode='markers', 
                                    marker=dict(size=5, color='red')))
        fig.update_layout(width=480, height=240)
        data = self.data[self.data.y==10]
        fig.add_trace(go.Scatter(x=data.x, 
                                 y=data.z, 
                                 line=dict(color='black')))
        fig.update_xaxes(title='J')
        fig.update_yaxes(title='Cp')
        fig.show()
    
    def x_vals(self, begin=0, end=4.8, dense=False):
        vals = np.linspace(0, 6, 6001) if dense else np.linspace(0, 6, 61)
        vals = np.round(vals, 3)
        return vals[ (vals >= begin) & (vals <= end) ]
    
    def y_vals(self, begin=10):
        vals = np.linspace(10, 60, 51)
        vals = np.round(vals, 2)
        return vals[vals >= begin]
    
    def z_vals(self, end=0.8):
        return np.linspace(0, end, 101)
        
    def create_data(self):
        points = self.extra[self.extra.y == 10]
        z = interp1d(points.x, points.z, kind='quadratic')
        x = self.x_vals(dense=True, end=points.x.max())
        x = np.append(x, points.x.max())
        self.data = self.appendTo(self.data, x, 10, z(x))
    
    

In [4]:
class Cp(Propeller):
    
    def densing(self):
        top = pd.DataFrame()
        extrapolate = False if hasattr(self, 'top') else True
        x_vals = [0, 0.3] if extrapolate else self.x_vals(end=3)
        mesh = self.mesh[self.mesh.x >= 10]
        df = self.data.append(mesh, ignore_index=True)
        for x in x_vals:
            inters = df[df.x == x]
            y_vals = self.y_vals(begin=10 if extrapolate else inters.y.min())
            kind = 'linear' if extrapolate else 'quadratic'
            z = interp1d(inters.y, inters.z, kind=kind, fill_value='extrapolate')
            top = self.appendTo(top, x, y_vals, z(y_vals))
        self.top = top
        
    def interpolate(self, points, mesh, z):
        x = interp1d(points.z, points.x, kind = 'quadratic', fill_value='extrapolate')
        y = interp1d(points.z, points.y, kind = 'quadratic', fill_value='extrapolate')
        mesh = mesh.append({'x': float(x(z)), 
                            'y': float(y(z)), 
                            'z': z}, ignore_index=True)
        return mesh
        
    def plane_intercepts(self, z):
        inters = pd.DataFrame()
        for y in self.data.y.unique():
            df = self.data[(self.data.y == y) & (self.data.z < 0.2)]
            inters = self.interpolate(df, inters, z)
        return inters
     
    def bottom(self, z_vals=[0]) -> pd.DataFrame:
        mesh = pd.DataFrame()
        for z in z_vals:
            inters = self.plane_intercepts(z)
            # y direction
            y_vals = self.y_vals()
            x = interp1d(inters.y, inters.x, kind='quadratic', fill_value='extrapolate')
            mesh = self.appendTo(mesh, x(y_vals), y_vals, z)
            # x direction
            x_vals = self.x_vals(end=5.5)
            y = interp1d(inters.x, inters.y, kind='quadratic', fill_value='extrapolate')
            mesh = self.appendTo(mesh, x_vals, y(x_vals), z)
        self.mesh = mesh
        
    def extrapolated_data(self):
        self.extra = self.mesh.append(self.top, ignore_index=True)
        
    def create_curves(self, append=False):
        curves = pd.DataFrame()
        df = self.top.append(self.mesh, ignore_index=True)
        mesh_x = self.mesh[self.mesh.y.isin(self.y_vals())].x
        x_vals = np.linspace(0, mesh_x[0], 10, endpoint=False)
        x_vals = np.append(x_vals, mesh_x)
        for y in prop.y_vals(begin=10):
            inters = df[df.y == y]
            z = interp1d(inters.x, inters.z, kind='quadratic')
            x = x_vals[x_vals <= inters.x.max()]
            curves = self.appendTo(curves, x, y, z(x))
        self.curves = curves.round({'x': 3, 'z': 4}).drop_duplicates(subset=['x', 'y'])

In [5]:
prop = Cp(spinner=False, blades=3)
prop.bottom(z_vals=[0])
prop.densing()
prop.extrapolated_data()
prop.create_data()
prop.fit()
prop.densing()

prop.draw(y_max=60)

In [7]:
prop.create_curves()
prop.save_data()

In [8]:
prop.draw_curves(y_max=60, top=False)

In [90]:
prop.draw_surface()